In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from ebnmpy.estimators import estimators
import pandas as pd

In [ ]:
def sample_point_normal(n, pi0=.9, mu=0, sigma=2):
    not_delta = stats.bernoulli.rvs(pi0, size=n) == 0
    z = np.full(n, mu, dtype=float)
    z[not_delta] = stats.norm.rvs(mu, sigma, size=not_delta.sum())
    return z

In [ ]:
def sample_point_t(n, pi0=.8, df=5, scale=1.5):
    not_delta = stats.bernoulli.rvs(pi0, size=n) == 0
    z = np.zeros(n)
    z[not_delta] = stats.t.rvs(df=df, scale=scale, size=not_delta.sum())
    return z

In [ ]:
def sample_assymetric_tophat(n, pi0=.5, a=-5, b=10):
    not_delta = stats.bernoulli.rvs(pi0, size=n) == 0
    z = np.zeros(n)
    z[not_delta] = stats.uniform.rvs(a, b - a, size=not_delta.sum())
    return z

In [ ]:
def get_rmse(theta, theta_hat):
    return np.sqrt(np.mean((theta_hat - theta) ** 2))

In [ ]:
def get_clcov(theta, samples, intervals=(.05, .95)):
    lower = np.quantile(samples, intervals[0], axis=0)
    upper = np.quantile(samples, intervals[1], axis=0)
    return ((theta >= lower) & (theta < upper)).mean()

In [ ]:
from tqdm import tqdm

In [ ]:
s = 1
n = 1000
n_posterior_samples = 1001
n_simulations = 10

In [ ]:
np.random.seed(0)

In [ ]:
samplers = {
    "Point-normal": sample_point_normal,
    "Point-t": sample_point_t,
    "Asymmetric tophat": sample_assymetric_tophat,
}

results = []

for _ in tqdm(range(n_simulations)):
    for sampler_name, sampler in samplers.items():
        theta = sampler(n)
        x = theta + stats.norm.rvs(size=n)

        for cls_name, cls in estimators.items():
            est = cls(include_posterior_sampler=True).fit(x=x, s=s)
            samples = est.sample(n_posterior_samples)

            loglik = est.log_likelihood_
            rmse = get_rmse(theta, theta_hat=est.posterior_["mean"])
            clcov = get_clcov(theta, samples)

            results.append((sampler_name, cls.__name__, loglik, rmse, clcov))

In [ ]:
cls = estimators["normal"]

In [ ]:
est = cls(include_posterior_sampler=True, mode="estimate").fit(x=x, s=s)
samples = est.sample(n_posterior_samples)